In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

In [2]:
from pyomo.environ import (
    Constraint,
    Var,
    ConcreteModel,
    Expression,
    Objective,
    TransformationFactory,
    value,
    units as pyunits,
)
from pyomo.network import Arc

from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties import (
    GenericParameterBlock,
    GenericReactionParameterBlock,
)
from idaes.models.unit_models import Feed, Mixer, Heater, PFR, Product

from idaes.core.solvers import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state

In [3]:
import egprod_ideal as thermo_props
import egprod_reaction as reaction_props

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

In [5]:
m.fs.thermo_params = GenericParameterBlock(**thermo_props.config_dict)
m.fs.reaction_params = GenericReactionParameterBlock(
    property_package=m.fs.thermo_params, **reaction_props.config_dict
)

In [6]:
m.fs.OXIDE = Feed(property_package=m.fs.thermo_params)
m.fs.ACID = Feed(property_package=m.fs.thermo_params)
m.fs.PROD = Product(property_package=m.fs.thermo_params)
m.fs.M101 = Mixer(
    property_package=m.fs.thermo_params, inlet_list=["reagent_feed", "catalyst_feed"]
)
m.fs.H101 = Heater(
    property_package=m.fs.thermo_params,
    has_pressure_change=False,
    has_phase_equilibrium=False,
)

In [7]:
m.fs.R101 = PFR(
    property_package=m.fs.thermo_params,
    reaction_package=m.fs.reaction_params,
    has_equilibrium_reactions=False,
    has_heat_of_reaction=True,
    has_heat_transfer=True,
    has_pressure_change=False,
    transformation_method="dae.finite_difference",
    transformation_scheme="BACKWARD",
    finite_elements=20,
)

In [8]:
m.fs.s01 = Arc(source=m.fs.OXIDE.outlet, destination=m.fs.M101.reagent_feed)
m.fs.s02 = Arc(source=m.fs.ACID.outlet, destination=m.fs.M101.catalyst_feed)
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.PROD.inlet)

In [9]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [10]:
m.fs.eg_prod = Expression(
    expr=pyunits.convert(
        m.fs.PROD.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]
        * m.fs.thermo_params.ethylene_glycol.mw,  # MW defined in properties as kg/mol
        to_units=pyunits.Mlb / pyunits.yr,
    )
)  # converting kg/s to MM lb/year

In [11]:
m.fs.cooling_cost = Expression(
    expr=2.12e-8
    * (-sum(m.fs.R101.heat_duty[0, x] for x in m.fs.R101.control_volume.length_domain))
)  # the reaction is exothermic, so R101 duty is negative
m.fs.heating_cost = Expression(
    expr=2.2e-7 * m.fs.H101.heat_duty[0]
)  # the stream must be heated to T_rxn, so H101 duty is positive
m.fs.operating_cost = Expression(
    expr=(3600 * 8000 * (m.fs.heating_cost + m.fs.cooling_cost))
)

In [12]:
print(degrees_of_freedom(m))

35


In [13]:
m.fs.OXIDE.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"].fix(
    58.0 * pyunits.mol / pyunits.s
)
m.fs.OXIDE.outlet.flow_mol_phase_comp[0, "Liq", "water"].fix(
    39.6 * pyunits.mol / pyunits.s
)  # calculated from 16.1 mol EO / cudm in stream
m.fs.OXIDE.outlet.flow_mol_phase_comp[0, "Liq", "sulfuric_acid"].fix(
    1e-5 * pyunits.mol / pyunits.s
)
m.fs.OXIDE.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(
    1e-5 * pyunits.mol / pyunits.s
)
m.fs.OXIDE.outlet.temperature.fix(298.15 * pyunits.K)
m.fs.OXIDE.outlet.pressure.fix(1e5 * pyunits.Pa)

m.fs.ACID.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"].fix(
    1e-5 * pyunits.mol / pyunits.s
)
m.fs.ACID.outlet.flow_mol_phase_comp[0, "Liq", "water"].fix(
    200 * pyunits.mol / pyunits.s
)
m.fs.ACID.outlet.flow_mol_phase_comp[0, "Liq", "sulfuric_acid"].fix(
    0.334 * pyunits.mol / pyunits.s
)  # calculated from 0.9 wt% SA in stream
m.fs.ACID.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(
    1e-5 * pyunits.mol / pyunits.s
)
m.fs.ACID.outlet.temperature.fix(298.15 * pyunits.K)
m.fs.ACID.outlet.pressure.fix(1e5 * pyunits.Pa)

In [14]:
m.fs.H101.outlet.temperature.fix(328.15 * pyunits.K)

In [15]:
m.fs.R101.conversion = Var(
    bounds=(0, 1), initialize=0.80, units=pyunits.dimensionless
)  # fraction

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion
    * m.fs.R101.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"]
    == (
        m.fs.R101.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"]
        - m.fs.R101.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"]
    )
)

for x in m.fs.R101.control_volume.length_domain:
    if x == 0:
        continue
    m.fs.R101.control_volume.properties[0, x].temperature.fix(
        328.15 * pyunits.K
    )  # equal inlet reactor temperature

m.fs.R101.conversion.fix(0.5)

m.fs.R101.length.fix(1 * pyunits.m)

In [16]:
m.fs.R101.heat_duty.setub(
    0 * pyunits.J / pyunits.s
)  # heat duty is only used for cooling

In [17]:
print(degrees_of_freedom(m))

0


In [18]:
# Initialize and solve each unit operation
m.fs.OXIDE.initialize()
propagate_state(arc=m.fs.s01)

m.fs.ACID.initialize()
propagate_state(arc=m.fs.s01)

m.fs.M101.initialize()
propagate_state(arc=m.fs.s03)

m.fs.H101.initialize()
propagate_state(arc=m.fs.s04)

m.fs.R101.initialize()
propagate_state(arc=m.fs.s05)

m.fs.PROD.initialize()

# set solver
solver = get_solver()

2023-02-18 12:05:44 [INFO] idaes.init.fs.OXIDE.properties: Starting initialization


2023-02-18 12:05:44 [INFO] idaes.init.fs.OXIDE.properties: Property initialization: optimal - Optimal Solution Found.


2023-02-18 12:05:44 [INFO] idaes.init.fs.OXIDE.properties: Property package initialization: optimal - Optimal Solution Found.


2023-02-18 12:05:44 [INFO] idaes.init.fs.OXIDE: Initialization Complete.


2023-02-18 12:05:44 [INFO] idaes.init.fs.ACID.properties: Starting initialization


...

2023-02-18 12:05:45 [INFO] idaes.init.fs.R101: Initialization Complete: optimal - Optimal Solution Found


2023-02-18 12:05:45 [INFO] idaes.init.fs.PROD.properties: Starting initialization


2023-02-18 12:05:46 [INFO] idaes.init.fs.PROD.properties: Property initialization: optimal - Optimal Solution Found.


2023-02-18 12:05:46 [INFO] idaes.init.fs.PROD.properties: Property package initialization: optimal - Optimal Solution Found.


2023-02-18 12:05:46 [INFO] idaes.init.fs.PROD: Initialization Complete.


In [19]:
# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [20]:
print(f"operating cost = ${value(m.fs.operating_cost)/1e6:0.3f} million per year")

operating cost = $43.176 million per year


In [21]:
m.fs.R101.report()

print()
print(f"Conversion achieved = {value(m.fs.R101.conversion):.1%}")
print()
print(
    f"Total heat duty required = "
    f"{value(sum(m.fs.R101.heat_duty[0, x] for x in m.fs.R101.control_volume.length_domain))/1e6:0.3f}"
    f" MJ"
)
print()
print(f"Tube area required = {value(m.fs.R101.area):0.3f} m^2")
print()
print(f"Tube length required = {value(m.fs.R101.length):0.3f} m")
print()
print(f"Tube volume required = {value(m.fs.R101.volume):0.3f} m^3")


Unit : fs.R101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key  : Value  : Units      : Fixed : Bounds
    Area : 1.1490 : meter ** 2 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                  Units         Inlet     Outlet  
    Molar Flowrate ('Liq', 'ethylene_oxide')   mole / second     58.000     29.000
    Molar Flowrate ('Liq', 'water')            mole / second     239.60     210.60
    Molar Flowrate ('Liq', 'sulfuric_acid')    mole / second    0.33401    0.33401
    Molar Flowrate ('Liq', 'ethylene_glycol')  mole / second 2.0000e-05     29.000
    Temperature                                       kelvin     328.15     328.15
    Pressure                                          pascal 1.0000e+05 1.0000e+05

Con

In [22]:
m.fs.objective = Objective(expr=m.fs.operating_cost)

In [23]:
m.fs.eg_prod_con = Constraint(
    expr=m.fs.eg_prod >= 200 * pyunits.Mlb / pyunits.yr
)  # MM lb/year
m.fs.R101.conversion.fix(0.90)

m.fs.R101.volume.setlb(0 * pyunits.m**3)
m.fs.R101.volume.setub(pyunits.convert(5000 * pyunits.gal, to_units=pyunits.m**3))

m.fs.R101.length.unfix()
m.fs.R101.length.setlb(0 * pyunits.m)
m.fs.R101.length.setub(5 * pyunits.m)

m.fs.H101.outlet.temperature.unfix()
m.fs.H101.outlet.temperature[0].setlb(328.15 * pyunits.K)
m.fs.H101.outlet.temperature[0].setub(
    470.45 * pyunits.K
)  # highest component boiling point (ethylene glycol)

for x in m.fs.R101.control_volume.length_domain:
    if x == 0:
        continue
    m.fs.R101.control_volume.properties[
        0, x
    ].temperature.unfix()  # allow for temperature change in each finite element

In [24]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

  12  1.5542747e+07 5.73e+03 2.72e+06  -1.0 2.39e+06    -  1.86e-01 9.96e-01h  1
  13  1.5546208e+07 1.34e+03 1.34e+06  -1.0 8.40e+05    -  8.89e-01 8.63e-01h  1
  14  1.5545842e+07 2.74e+03 6.38e+05  -1.0 2.16e+06    -  7.32e-01 8.70e-01f  1
  15  1.5545565e+07 2.08e+03 5.06e+05  -1.0 8.20e+02  -4.0 9.91e-01 2.40e-01f  1
  16  1.5544345e+07 8.68e+00 2.21e+05  -1.0 8.77e+02  -4.5 1.00e+00 1.00e+00f  1
  17  1.5542854e+07 6.76e+00 8.02e+04  -1.0 1.12e+03  -5.0 1.00e+00 1.00e+00f  1
  18  1.5541120e+07 9.10e+00 1.75e+04  -1.0 1.33e+03  -5.4 1.00e+00 1.00e+00f  1
  19  1.5539121e+07 1.23e+01 1.10e+04  -1.0 1.73e+03  -5.9 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  1.5538942e+07 1.14e+01 1.03e+04  -1.0 4.96e+03  -6.4 1.00e+00 8.43e-02f  1
  21  1.5538940e+07 1.14e+01 1.03e+04  -1.0 6.09e+03  -6.9 3.12e-01 8.54e-04f  1
  22  1.5538939e+07 1.12e+01 9.65e+03  -1.0 1.47e+06  -7.3 1.30e-04 1.47e-02f  1
  23  1.5538912e+07 2.70e+04

  29  1.5538912e+07 2.04e+05 8.46e+01  -1.0 1.87e+07    -  5.16e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.5538912e+07 1.59e+05 3.47e+01  -1.0 1.59e+07    -  9.86e-01 1.00e+00h  1
  31  1.5538912e+07 1.24e+05 2.08e+01  -1.0 9.46e+06    -  2.25e-01 2.38e-01H  1
  32  1.5538912e+07 5.12e+04 8.85e+00  -1.0 8.55e+06    -  1.00e+00 1.00e+00f  1
  33  1.5538912e+07 6.37e+03 2.36e+00  -1.0 2.94e+06    -  1.00e+00 1.00e+00h  1
  34  1.5538912e+07 1.11e+03 4.70e-01  -1.0 1.25e+06    -  1.00e+00 1.00e+00h  1
  35  1.5538912e+07 3.25e+00 3.62e-02  -1.7 6.61e+04    -  1.00e+00 1.00e+00h  1
  36  1.5538912e+07 6.41e-03 5.57e-04  -3.8 3.02e+03    -  1.00e+00 1.00e+00h  1
  37  1.5538912e+07 3.46e-05 4.13e-07  -7.0 2.22e+02    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 37

                                   (scaled)                 (unscaled)
Objective...............:   1.5538911764822479e+07    1.5538911764822479e+07
Dual infeas

In [25]:
print(f"operating cost = ${value(m.fs.operating_cost)/1e6:0.3f} million per year")

print()
print("Heater results")

m.fs.H101.report()

print()
print("PFR reactor results")

m.fs.R101.report()

operating cost = $15.539 million per year

Heater results

Unit : fs.H101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value  : Units : Fixed : Bounds
    Heat Duty : 699.26 :  watt : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                  Units         Inlet     Outlet  
    Molar Flowrate ('Liq', 'ethylene_oxide')   mole / second     58.000     58.000
    Molar Flowrate ('Liq', 'water')            mole / second     239.60     239.60
    Molar Flowrate ('Liq', 'sulfuric_acid')    mole / second    0.33401    0.33401
    Molar Flowrate ('Liq', 'ethylene_glycol')  mole / second 2.0000e-05 2.0000e-05
    Temperature                                       kelvin     298.15     328.15
    Pressure                 

In [26]:
print("Optimal Values")
print()

print(f"H101 outlet temperature = {value(m.fs.H101.outlet.temperature[0]):0.3f} K")

print()
print(
    "Total heat duty required = ",
    value(
        sum(m.fs.R101.heat_duty[0, x] for x in m.fs.R101.control_volume.length_domain)
    )
    / 1e6,
    "MJ",
)
print()
print(f"Tube area required = {value(m.fs.R101.area):0.3f} m^2")

print()
print(f"Tube length required = {value(m.fs.R101.length):0.3f} m")

print()
print(
    f"Assuming a 20% design factor for reactor volume,"
    f"total CSTR volume required = {value(1.2*m.fs.R101.volume):0.3f}"
    f" m^3 = {value(pyunits.convert(1.2*m.fs.R101.volume, to_units=pyunits.gal)):0.3f} gal"
)

print()
print(f"Ethylene glycol produced = {value(m.fs.eg_prod):0.3f} MM lb/year")

print()
print(f"Conversion achieved = {value(m.fs.R101.conversion):.1%}")

Optimal Values

H101 outlet temperature = 328.150 K

Total heat duty required =  -25.443005245978185 MJ

Tube area required = 2.787 m^2

Tube length required = 5.000 m

Assuming a 20% design factor for reactor volume,total CSTR volume required = 16.725 m^3 = 4418.148 gal

Ethylene glycol produced = 225.415 MM lb/year

Conversion achieved = 90.0%
